In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText

###############################################################################
# 1) Initialize Model & Processor from the local /scratch folder
###############################################################################
model_path = "qwen_model"

processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)

# Define examples
examples = [
    {
        {
        "seller_description": """
        exmaple product seller desciption
        """,
        "buyer_reviews": [
          """  Review-1)...
          Review-2)...
          ..
          .
          .
          
""" 
     
        ],
        "expected_output": """
       example output
      
        """
    },
    {
        "seller_description": """
        exmaple product seller desciption
        """,
        "buyer_reviews": [
          """  Review-1)...
          Review-2)...
          ..
          .
          .
          
""" 
     
        ],
        "expected_output": """
       example output
      
        """
    },
    {
        "seller_description": """
        exmaple product seller desciption
        """,
        "buyer_reviews": [
          """  Review-1)...
          Review-2)...
          ..
          .
          .
          
""" 
     
        ],
        "expected_output": """
       example output
      
        """
    },
]

# New task for the model to analyze
new_seller_description = """
seller product description

""",

new_buyer_reviews = [
    """Review-1) ....
    Review-2) ........
    .
    .
    .
    .
    .
    
    
"""
system_instruction = """
LoT prompt
"""

###############################################################################
# 3) Function to Build the Prompt
###############################################################################
def build_prompt(examples, seller_desc, buyer_revs):
    """
    Builds a single text prompt that includes:
      1) The system instruction
      2) The 'Examples' section (with seller desc, buyer reviews, and expected outputs)
      3) The 'New Task' with the new seller description and new buyer reviews
      4) A final line instructing the model to produce a 'Structured Review Analysis with Counts'
    """
    prompt = f"{system_instruction}\n\nExamples:\n\n"

    # Add each example’s content
    for example in examples:
        prompt += f"Seller's Description:\n{example['seller_description']}\n\nBuyer Reviews:"
        for i, review in enumerate(example['buyer_reviews'], 1):
            prompt += f"\nReview {i}: {review}"
        prompt += f"\n\nExpected Output:\n{example['expected_output']}\n\n"

    # Add the new task
    prompt += "New Task:\n"
    prompt += f"Seller's Description:\n{seller_desc}\n\nBuyer Reviews:"
    for i, review_text in enumerate(buyer_revs, 1):
        prompt += f"\nReview {i}: {review_text}"

    # Instruct the model to output the structured review analysis
    prompt += "\n\nStructured Review Analysis with Counts:"
    return prompt

###############################################################################
# 4) Build the Prompt
###############################################################################
prompt = build_prompt(examples, new_seller_description, new_buyer_reviews)

###############################################################################
# 5) Prepare Model Inputs (Text Only)
###############################################################################
inputs = processor(
    text=[prompt],
    return_tensors="pt",
    padding=True
).to("cuda" if torch.cuda.is_available() else "cpu")

###############################################################################
# 6) Generate the Model Output
###############################################################################
generated_ids = model.generate(
    **inputs,
    max_new_tokens=3000,       
    temperature=0.7,           
    repetition_penalty=1.2,    
    top_k=50,                  
    top_p=0.9                  
)

###############################################################################
# 7) Decode the Output
###############################################################################
output_text = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

###############################################################################
# 8) Print the Final Analysis
###############################################################################
print("Generated Structured Analysis with Counts:\n", output_text[0])